<a href="https://colab.research.google.com/github/RushiBShinde/ThinkForge-IE-643-project/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers sentence-transformers langchain faiss-cpu accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# ==============================================================================
# 1. INSTALL LIBRARIES
# ==============================================================================
# We need several libraries:
# - transformers & sentence-transformers: For loading our LLM and embedding model.
# - langchain: A popular framework that simplifies building RAG applications.
# - faiss-cpu: A library for efficient similarity search, which acts as our vector store.
# - accelerate & bitsandbytes: To load our LLM efficiently.

#!pip install -q transformers sentence-transformers langchain faiss-cpu accelerate bitsandbytes

# ==============================================================================
# 2. DEFINE YOUR DOCUMENTS
# ==============================================================================
# In a real-world scenario, you would load your own data (PDFs, TXTs, etc.).
# For this example, we'll just create a few simple text documents in memory.

from langchain.docstore.document import Document

# Create a list of Document objects
documents = [
    Document(page_content="The Wright brothers made the first sustained flight in 1903."),
    Document(page_content="The Apollo 11 mission was the first to land humans on the Moon in 1969."),
    Document(page_content="The International Space Station (ISS) is a habitable artificial satellite in low Earth orbit."),
    Document(page_content="A black hole is a region of spacetime where gravity is so strong that nothing, no particles or even electromagnetic radiation such as light, can escape from it."),
    Document(page_content="T5 follows a simple yet effective principle i.e it convert all NLP problems into a text-to-text format. Model uses encoder-decoder architecture similar to Transformer-based sequence-to-sequence models. It works by :Task Formulation as Text-to-Text: Instead of treating different NLP tasks separately it reformulates each problem into a text-based input and output. Encoding the Input: The input text is tokenized using SentencePiece, then passed through the encoder which generates a contextual representation. Decoding the Output: The decoder takes the encoded representation and generates the output text in a autoregressive manner. Training the Model: T5 is pre-trained using a denoising objective where portions of text are masked and the model learns to reconstruct them. It is then fine-tuned for various tasks.")
]

print("--- Documents Created ---")
print(documents)









--- Documents Created ---
[Document(metadata={}, page_content='The Wright brothers made the first sustained flight in 1903.'), Document(metadata={}, page_content='The Apollo 11 mission was the first to land humans on the Moon in 1969.'), Document(metadata={}, page_content='The International Space Station (ISS) is a habitable artificial satellite in low Earth orbit.'), Document(metadata={}, page_content='A black hole is a region of spacetime where gravity is so strong that nothing, no particles or even electromagnetic radiation such as light, can escape from it.'), Document(metadata={}, page_content='T5 follows a simple yet effective principle i.e it convert all NLP problems into a text-to-text format. Model uses encoder-decoder architecture similar to Transformer-based sequence-to-sequence models. It works by :Task Formulation as Text-to-Text: Instead of treating different NLP tasks separately it reformulates each problem into a text-based input and output. Encoding the Input: The inpu

In [ ]:
# ==============================================================================
# 3. CREATE THE VECTOR STORE (INDEXING)
# ==============================================================================
# This is the core of the "Retrieval" part. We are creating a searchable
# database for our documents.

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Load an embedding model
# This model is responsible for turning our text into numerical vectors.
# 'all-MiniLM-L6-v2' is a small, fast, and popular choice.
print("\n--- Loading Embedding Model ---")
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Create the vector store using FAISS
# The from_documents method handles everything for us:
# 1. It takes our documents.
# 2. It uses the embedding model to turn them into vectors.
# 3. It stores these vectors in a FAISS index for fast retrieval.
print("\n--- Creating Vector Store ---")
vector_store = FAISS.from_documents(documents, embeddings)
print("--- Vector Store Ready! ---")


--- Loading Embedding Model ---

--- Creating Vector Store ---
--- Vector Store Ready! ---


In [ ]:
# ==============================================================================
# 4. SETUP THE LANGUAGE MODEL (LLM)
# ==============================================================================
# This is the "Generation" part of RAG. We'll use a powerful but manageable
# model from Google called Flan-T5.

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load the tokenizer and model
model_id = 'google/flan-t5-base' # A good, medium-sized model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

# We wrap the model and tokenizer in a HuggingFacePipeline for easy use with LangChain
llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    torch_dtype=torch.bfloat16
)

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)

print("\n--- LLM Ready! ---")
print(llm.invoke("What is the capital of France?")) # Test the LLM on its own


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
/tmp/ipython-input-4088951206.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)



--- LLM Ready! ---
london


In [ ]:
# ==============================================================================
# 5. CREATE THE RAG CHAIN
# ==============================================================================
# Now we'll chain everything together. LangChain Expression Language (LCEL)
# makes this very intuitive. The chain will:
# 1. Take a user's question.
# 2. Find relevant documents in the vector store (this is the retriever).
# 3. Format a prompt with the question and the retrieved documents.
# 4. Pass the prompt to the LLM to get an answer.

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# First, get a retriever from our vector store. A retriever is a component
# that "retrieves" documents. `k=2` means it will fetch the top 2 most relevant documents.
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Create a prompt template. This will be filled with the retrieved context and the user's question.
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

# Now, create the RAG chain using LCEL syntax
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("\n--- RAG Chain Ready! ---")



--- RAG Chain Ready! ---


In [ ]:
# ==============================================================================
# 6. ASK QUESTIONS!
# ==============================================================================
# Now we can invoke our chain with a question. The RAG system will use the
# documents we provided to answer, not its general pre-trained knowledge.

print("\n--- Asking Questions using RAG ---")

# Question 1: Related to our documents
question1 = "When did humans first land on the moon?"
print(f"Question: {question1}")
response1 = rag_chain.invoke(question1)
print(f"Answer: {response1}")

# # Question 2: Also related to our documents
# question2 = "What is a feature of the International Space Station?"
# print(f"\nQuestion: {question2}")
# response2 = rag_chain.invoke(question2)
# print(f"Answer: {response2}")

# Question 3: Not in our documents. The model should say it doesn't know.
question3 = "How does encoding the input work in T5?"
print(f"\nQuestion: {question3}")
response3 = rag_chain.invoke(question3)
print(f"Answer: {response3}")




--- Asking Questions using RAG ---
Question: When did humans first land on the moon?
Answer: 1969

Question: How does encoding the input work in T5?
Answer: The input text is tokenized using SentencePiece, then passed through the encoder which generates a contextual representation. Decoding the Output: The decoder takes the encoded representation and generates the output text in a autoregressive manner. Training the Model: T5 is pre-trained using a denoising objective where portions of text are masked and the model learns to reconstruct them. It is then fine-tuned for various tasks.'
